Segunda célula: Configurando e compilando o código CUDA

O código em CUDA para multiplicar matrizes e medir o tempo de execução

In [7]:
%%writefile multiplicacao_matriz_gpu.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>

// Função para multiplicação de matrizes no GPU
__global__ void multiplicarMatrizes(int* matriz1, int* matriz2, int* resultado, int tamanho) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < tamanho && col < tamanho) {
        int sum = 0;
        for (int i = 0; i < tamanho; i++) {
            sum += matriz1[row * tamanho + i] * matriz2[i * tamanho + col];
        }
        resultado[row * tamanho + col] = sum;
    }
}

int main() {
    int tamanho_matriz;

    // Solicitar o tamanho da matriz quadrada
    printf("Digite o tamanho da matriz quadrada: ");
    if (scanf("%d", &tamanho_matriz) != 1) {
        printf("Erro ao ler o tamanho da matriz.\n");
        return 1;
    }

    // Alocar memória no host (CPU)
    int* h_matriz1 = (int*)malloc(tamanho_matriz * tamanho_matriz * sizeof(int));
    int* h_matriz2 = (int*)malloc(tamanho_matriz * tamanho_matriz * sizeof(int));
    int* h_resultado = (int*)malloc(tamanho_matriz * tamanho_matriz * sizeof(int));

    // Preencher as matrizes com valores aleatórios
    srand(time(NULL));
    for (int i = 0; i < tamanho_matriz * tamanho_matriz; i++) {
        h_matriz1[i] = rand() % 10;
        h_matriz2[i] = rand() % 10;
    }

    // Alocar memória no device (GPU)
    int* d_matriz1;
    int* d_matriz2;
    int* d_resultado;
    cudaMalloc(&d_matriz1, tamanho_matriz * tamanho_matriz * sizeof(int));
    cudaMalloc(&d_matriz2, tamanho_matriz * tamanho_matriz * sizeof(int));
    cudaMalloc(&d_resultado, tamanho_matriz * tamanho_matriz * sizeof(int));

    // Copiar os dados do host (CPU) para o device (GPU)
    cudaMemcpy(d_matriz1, h_matriz1, tamanho_matriz * tamanho_matriz * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_matriz2, h_matriz2, tamanho_matriz * tamanho_matriz * sizeof(int), cudaMemcpyHostToDevice);

    // Definir o número de threads e blocos
    dim3 threadsPorBloco(16, 16);
    dim3 blocosPorGrade((tamanho_matriz + threadsPorBloco.x - 1) / threadsPorBloco.x,
                        (tamanho_matriz + threadsPorBloco.y - 1) / threadsPorBloco.y);

    // Medir o tempo de execução no GPU
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    // Executar o kernel para multiplicar as matrizes
    multiplicarMatrizes<<<blocosPorGrade, threadsPorBloco>>>(d_matriz1, d_matriz2, d_resultado, tamanho_matriz);

    // Medir o tempo de execução
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    // Copiar o resultado de volta para o host (CPU)
    cudaMemcpy(h_resultado, d_resultado, tamanho_matriz * tamanho_matriz * sizeof(int), cudaMemcpyDeviceToHost);

    // Exibir o tempo de execução no GPU
    printf("Tempo de execução na GPU para multiplicar matrizes %dx%d: %f segundos\n", tamanho_matriz, tamanho_matriz, milliseconds / 1000);

    // Liberar memória
    cudaFree(d_matriz1);
    cudaFree(d_matriz2);
    cudaFree(d_resultado);
    free(h_matriz1);
    free(h_matriz2);
    free(h_resultado);

    return 0;
}

Overwriting multiplicacao_matriz_gpu.cu


In [8]:
!nvcc multiplicacao_matriz_gpu.cu -o multiplicacao_matriz_gpu

In [37]:
!./multiplicacao_matriz_gpu

Digite o tamanho da matriz quadrada: 20000
Tempo de execução na GPU para multiplicar matrizes 20000x20000: 68.102074 segundos


In [2]:
%%writefile multiplicacao_matriz_cpu.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

// Função para alocar memória para uma matriz
int** alocarMatriz(int linhas, int colunas) {
    int** matriz = (int**)malloc(linhas * sizeof(int*));
    for (int i = 0; i < linhas; i++) {
        matriz[i] = (int*)malloc(colunas * sizeof(int));
    }
    return matriz;
}

// Função para liberar a memória alocada para uma matriz
void liberarMatriz(int** matriz, int linhas) {
    for (int i = 0; i < linhas; i++) {
        free(matriz[i]);
    }
    free(matriz);
}

// Função para preencher uma matriz com valores aleatórios
void preencherMatriz(int** matriz, int linhas, int colunas) {
    for (int i = 0; i < linhas; i++) {
        for (int j = 0; j < colunas; j++) {
            matriz[i][j] = rand() % 10; // Valores aleatórios entre 0 e 9
        }
    }
}

// Função para multiplicar duas matrizes
int** multiplicarMatrizes(int** matriz1, int** matriz2, int linhas1, int colunas1, int colunas2) {
    int** resultado = alocarMatriz(linhas1, colunas2);

    for (int i = 0; i < linhas1; i++) {
        for (int j = 0; j < colunas2; j++) {
            resultado[i][j] = 0;
            for (int k = 0; k < colunas1; k++) {
                resultado[i][j] += matriz1[i][k] * matriz2[k][j];
            }
        }
    }

    return resultado;
}

int main() {
    int tamanho_matriz;

    // Solicitar o tamanho da matriz quadrada
    printf("Digite o tamanho da matriz quadrada: ");
    if (scanf("%d", &tamanho_matriz) != 1) {
        printf("Erro ao ler o tamanho da matriz.\n");
        return 1;
    }

    srand(time(NULL)); // Inicializa a semente para números aleatórios

    // Alocar e preencher as matrizes
    int** matriz1 = alocarMatriz(tamanho_matriz, tamanho_matriz);
    int** matriz2 = alocarMatriz(tamanho_matriz, tamanho_matriz);
    preencherMatriz(matriz1, tamanho_matriz, tamanho_matriz);
    preencherMatriz(matriz2, tamanho_matriz, tamanho_matriz);

    // Medir o tempo de execução da multiplicação
    clock_t inicio = clock(); // Pega o tempo antes da multiplicação

    // Multiplicar as matrizes
    int** resultado = multiplicarMatrizes(matriz1, matriz2, tamanho_matriz, tamanho_matriz, tamanho_matriz);

    clock_t fim = clock(); // Pega o tempo após a multiplicação
    double tempo_execucao = (double)(fim - inicio) / CLOCKS_PER_SEC; // Calcula o tempo em segundos

    // Exibe o tempo de execução
    printf("Tempo de execução na CPU para multiplicar matrizes %dx%d: %f segundos\n", tamanho_matriz, tamanho_matriz, tempo_execucao);

    // Liberar a memória alocada
    liberarMatriz(matriz1, tamanho_matriz);
    liberarMatriz(matriz2, tamanho_matriz);
    liberarMatriz(resultado, tamanho_matriz);

    return 0;
}

Writing multiplicacao_matriz_cpu.c


In [3]:
!gcc multiplicacao_matriz_cpu.c -o multiplicacao_matriz_cpu -O2

In [6]:
!./multiplicacao_matriz_cpu

Digite o tamanho da matriz quadrada: 500
Tempo de execução na CPU para multiplicar matrizes 500x500: 0.150315 segundos
